<a href="https://colab.research.google.com/github/coralie-sorbet/Enhancing-LLM-with-human-feedback/blob/main/Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **End-to-End Reinforcement Learning with Human Feedback: Reward Modeling and PPO Training**


## **Load Libraries and configuration of the models**


In [3]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from trl import (
    AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer,
    RewardConfig, RewardTrainer, setup_chat_format, ModelConfig
)
from torch.amp import autocast, GradScaler
from datasets import load_dataset

# Set model name and device (use GPU if available)
model_name = "gpt2"  # Base model name
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Initialize model configuration
model_config = ModelConfig(model_name_or_path=model_name)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(
    model_config.model_name_or_path,
    trust_remote_code=model_config.trust_remote_code,
    use_fast=True
)
tokenizer.pad_token = tokenizer.eos_token  # Ensure EOS token is used for padding

# Load models for different purposes
model = AutoModelForSequenceClassification.from_pretrained(
    model_config.model_name_or_path, num_labels=1, trust_remote_code=model_config.trust_remote_code
)
reward_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)
policy_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
ref_policy_model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
value_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pytorch_model.bin:   0%|          | 0.00/548M [00:00<?, ?B/s]

## **Data Preparation**

In [4]:
# Load and preprocess the ultrafeedback_binarized dataset
dataset = load_dataset("trl-lib/ultrafeedback_binarized")

# Data preparation: Truncate long sequences
def tokenize_function(examples):
    return tokenizer(
        [str(x) for x in examples["chosen"]],
        truncation=True,
        padding="max_length",
        max_length=min(tokenizer.model_max_length, 200), #Reduced length to 200 for CUDA memory purposes
        return_tensors="pt" # Added return_tensors to return PyTorch tensors
    )

# Tokenize dataset
tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    num_proc=4,
    # remove_columns=dataset["train"].column_names
)

README.md:   0%|          | 0.00/643 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/131M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/2.14M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/62135 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/62135 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

In [5]:
print(dataset) # Shows the features of the original dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
})


In [6]:
print(tokenized_dataset)  # Shows the features of the tokenized dataset

DatasetDict({
    train: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids', 'attention_mask'],
        num_rows: 62135
    })
    test: Dataset({
        features: ['chosen', 'rejected', 'score_chosen', 'score_rejected', 'input_ids', 'attention_mask'],
        num_rows: 1000
    })
})


## **Train the Reward Model**

In [ ]:
# Set up chat format for the tokenizer and model
if tokenizer.chat_template is None:
        model, tokenizer = setup_chat_format(model, tokenizer)

# Configure training arguments for the reward model
reward_config = RewardConfig(
    output_dir="./results",
    per_device_train_batch_size=8,
    num_train_epochs=1,
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=50,
    remove_unused_columns=False,  # Required for `RewardDataCollatorWithPadding`
    dataset_num_proc=4,
    report_to="none"
)

# Initialize the RewardTrainer
trainer = RewardTrainer(
        args=reward_config,
        model=model,
        tokenizer=tokenizer,
        train_dataset=tokenized_dataset["train"],
        eval_dataset=tokenized_dataset['test'],
    )

# Train the reward model
print("Training the reward model...")
trainer.train()

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Map:   0%|          | 0/62135 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/62135 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (1063 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1942 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1096 > 1024). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (2164 > 1024). Running this sequence through the model will result in indexing errors


Filter (num_proc=4):   0%|          | 0/62135 [00:00<?, ? examples/s]

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.4741, 0.5259] │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improve the operational performance of its   │                  │
│ reduce the number and severity of traffic     │ transportation network;   reduce the number  │                  │
│ collisions and increase driver, passenger,    │ and severity of traffic collisions and       │                  │
│ and pedestrian safety;  collect, disseminate, │ increase driver, passenger, and pedestrian   │                  │
│ and utilize real-time traffic, transit,       │ safety;  collect, disseminate, and utilize   │                  │
│ parking, and other transportation-related     │ real-time traffic, transit, parking, and     │                  │
│ information to improve mobility, reduce       │ other 

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:763: UserWarning: There are 310 out of 550 instances where the predictions for both options are equal. As a consequence the accuracy can be misleading.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.5081, 0.4919] │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improve the operational performance of its   │                  │
│ reduce the number and severity of traffic     │ transportation network;   reduce the number  │                  │
│ collisions and increase driver, passenger,    │ and severity of traffic collisions and       │                  │
│ and pedestrian safety;  collect, disseminate, │ increase driver, passenger, and pedestrian   │                  │
│ and utilize real-time traffic, transit,       │ safety;  collect, disseminate, and utilize   │                  │
│ parking, and other transportation-related     │ real-time traffic, transit, parking, and     │                  │
│ information to improve mobility, reduce       │ other 

/usr/local/lib/python3.10/dist-packages/trl/trainer/utils.py:763: UserWarning: There are 323 out of 550 instances where the predictions for both options are equal. As a consequence the accuracy can be misleading.
  warnings.warn(


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┓
┃ chosen_text                                   ┃ rejected_text                                ┃ logits           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━┩
│ <|im_start|>user                              │ <|im_start|>user                             │ [0.5215, 0.4785] │
│ Definition: In this task, you are given a     │ Definition: In this task, you are given a    │                  │
│ summary for US Congressional and California   │ summary for US Congressional and California  │                  │
│ state bill, your task is to generate a Title  │ state bill, your task is to generate a Title │                  │
│ for this bill. The preferred titles are under │ for this bill. The preferred titles are      │                  │
│ forty words and mention the purpose of the    │ under forty words and mention the purpose of │                  │
│ bill.                                         │ the bill.                                    │                  │
│ Input: Intelligent Technologies Initiative    │ Input: Intelligent Technologies Initiative   │                  │
│ Act of 2015  This bill directs the Department │ Act of 2015  This bill directs the           │                  │
│ of Transportation (DOT) to establish an       │ Department of Transportation (DOT) to        │                  │
│ Intelligent Technology Initiative to provide  │ establish an Intelligent Technology          │                  │
│ grants to up to six state and local           │ Initiative to provide grants to up to six    │                  │
│ governments to establish deployment sites for │ state and local governments to establish     │                  │
│ large scale installation and operation of     │ deployment sites for large scale             │                  │
│ intelligent transportation systems (ITS) to   │ installation and operation of intelligent    │                  │
│ improve safety, efficiency, system            │ transportation systems (ITS) to improve      │                  │
│ performance, and return on investment. DOT    │ safety, efficiency, system performance, and  │                  │
│ shall develop criteria for selecting grant    │ return on investment. DOT shall develop      │                  │
│ recipients, including how the deployment of   │ criteria for selecting grant recipients,     │                  │
│ technology will enable the recipient to:      │ including how the deployment of technology   │                  │
│ reduce costs and improve return on            │ will enable the recipient to:   reduce costs │                  │
│ investments;   deliver environmental benefits │ and improve return on investments;   deliver │                  │
│ and reduce energy consumption by alleviating  │ environmental benefits and reduce energy     │                  │
│ congestion and streamlining traffic flow;     │ consumption by alleviating congestion and    │                  │
│ measure and improve the operational           │ streamlining traffic flow;   measure and     │                  │
│ performance of its transportation network;    │ improve the operational performance of its   │                  │
│ reduce the number and severity of traffic     │ transportation network;   reduce the number  │                  │
│ collisions and increase driver, passenger,    │ and severity of traffic collisions and       │                  │
│ and pedestrian safety;  collect, disseminate, │ increase driver, passenger, and pedestrian   │                  │
│ and utilize real-time traffic, transit,       │ safety;  collect, disseminate, and utilize   │                  │
│ parking, and other transportation-related     │ real-time traffic, transit, parking, and     │                  │
│ information to improve mobility, reduce       │ other 

In [ ]:
# Evaluation after training
if reward_config.eval_strategy != "no":
    print("Evaluating the model...")
    metrics = trainer.evaluate()
    print("\n***** Evaluation Metrics *****")
    for key, value in metrics.items():
        print(f"{key}: {value}")
    trainer.log_metrics("eval", metrics)
    trainer.save_metrics("eval", metrics)

# Save the model
print("Reward model trained and saved.")
reward_model.save_pretrained("./reward_model")

### **Training Results**

#### **1. Processing Speed:**
   - **Before filtering**: **4047.86 examples/sec**.
   - **After filtering**: **1712.32 examples/sec**.

   The processing speed decreases after applying the filter, as expected, since the model is now working with cleaner and more refined data. This highlights an additional step in data preparation.

#### **2. Training Loss:**
   - At **step 50**, the training loss is **0.7056**.
   - By **step 100**, it slightly decreases to **0.6905**.
   - Subsequent steps show a **fluctuating pattern**, indicating some instability in the model’s learning process:
     - **Step 350**: **0.694**
     - **Step 400**: **0.692**
     - **Step 500**: **0.694**

   While the loss doesn’t show a significant downward trend, it appears to stabilize after the fluctuations. This suggests the model is gradually converging but may require further fine-tuning to optimize performance.

#### **3. Validation Loss:**
   - The validation loss remains **constant at 0.6931** throughout training, which indicates the model is **not generalizing well** to unseen data. This flat behavior may suggest overfitting or underfitting. The model is struggling to improve on the validation set, potentially due to:
     - Insufficient model complexity.
     - Lack of regularization.

#### **4. Accuracy:**
   - At **step 50**, accuracy is **0.736**.
   - By **step 250**, it increases slightly to **0.774**.
   - However, accuracy fluctuates significantly in later steps:
     - **Step 350**: drops to **0.738**.
     - **Step 400**: further decreases to **0.725**.
     - Despite spikes, such as reaching **0.8836** at **step 3500**, overall accuracy lacks consistency.

   These fluctuations indicate potential challenges in learning stability. The model may occasionally improve but struggles to maintain steady progress, hinting at overfitting to training data or suboptimal hyperparameters.

---

### **Evaluation Results**

1. **Evaluation Accuracy**: **84.73%**  
   This suggests the model correctly predicts a significant portion of evaluation samples. However, the relevance of this accuracy should be contextualized, e.g., compared against baseline performance (e.g., random guessing).

2. **Evaluation Loss**: **0.6931**  
   A loss value of **0.6931** is characteristic of cross-entropy loss when class probabilities are close to 0.5 (log(2)). This indicates the model struggles to differentiate between classes. Potential causes include class imbalance, overfitting, or insufficient training.

3. **Evaluation Runtime**: **33.08 seconds**  
   While this runtime reflects the evaluation duration for the dataset, profiling could help determine if the computational process (e.g., GPU usage, data loading) can be optimized.

4. **Samples Per Second**: **16.624** | **Steps Per Second**: **2.086**  
   These metrics provide insights into computational efficiency. If evaluation appears slow, potential bottlenecks may involve large batch sizes, inefficient pipelines, or hardware limitations.

---

### **Key Observations and Recommendations**

#### **1. Addressing Stagnant Loss (0.6931):**
If the loss remains stagnant throughout training, the following adjustments could help:

- **Learning Rate Adjustment**: Increase the learning rate or use a scheduler to dynamically adjust it.  
- **Optimizer Selection**: If using SGD, switch to adaptive optimizers like Adam to improve convergence.  
- **Regularization**: Add dropout or weight decay to reduce overfitting.  
- **Batch Size Tuning**: Experiment with smaller or larger batch sizes to balance stability and speed.  
- **Activation Functions**: Replace sigmoid/tanh with ReLU variants to mitigate vanishing gradients.  
- **Data Issues**: Verify that the data pipeline ensures proper shuffling and diversity in training samples.  

#### **2. Improving Generalization:**
The flat validation loss and fluctuating accuracy indicate overfitting or underfitting. Possible remedies include:
- **Regularization**: Techniques like dropout, batch normalization, or early stopping.  
- **Data Augmentation**: Expand the dataset with variations to improve robustness.  
- **Revised Architecture**: Experiment with deeper models or transfer learning using pre-trained architectures.  

---

### **Next Steps: Implementing Proximal Policy Optimization (PPO)**

To enhance training results, transitioning to **Proximal Policy Optimization (PPO)** is a logical next step. PPO’s ability to balance exploration and exploitation will stabilize the learning process and address the observed inconsistencies.

#### **Expected Benefits of PPO**:
- **Improved Accuracy**: Achieve a more consistent upward trend in performance.  
- **Better Generalization**: Reduce overfitting, enabling the model to perform better on unseen data.  
- **Stabilized Loss**: Mitigate fluctuations, ensuring steady progress during training.

By incorporating PPO, we aim to establish a more robust and reliable learning framework, ultimately yielding a model with improved performance metrics and enhanced stability.

---


## **Train the Policy Model with PPO**

The dataset tokenized earlier has max_lenth too high that my computer can handle for PPOtraining. We re-tokenize with a lower length.

In [ ]:
ppo_config = PPOConfig(
    num_train_epochs=1, # Setting to 1 epoch to reduce time run even if not the best precision
    gradient_accumulation_steps=2,  # Reduced gradient accumulation steps for memory purposes
    batch_size=1,
    mini_batch_size=1,
    learning_rate=1.41e-5,
    output_dir="./results",
    logging_steps=25,
    eval_strategy="steps",
    eval_steps=50,
    report_to="none"
)

# Use a smaller train dataset subset for testing
train_dataset = tokenized_dataset["train"].select(range(int(len(tokenized_dataset["train"]) * 0.1))) #Reduced test set to decrease the run time
train_dataset = train_dataset.with_format("torch", columns=['input_ids', 'attention_mask'])

eval_dataset = tokenized_dataset["test"].map(tokenize_function, batched=True, num_proc=4)
eval_dataset = eval_dataset.with_format("torch", columns=['input_ids', 'attention_mask']) # Ensure correct format

trainer = PPOTrainer(
    ppo_config,
    reward_model=reward_model,
    tokenizer=tokenizer,
    policy=policy_model,
    ref_policy=ref_policy_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    value_model=value_model,
)

trainer.train()

Thu Dec 12 09:16:00 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   55C    P0              29W /  70W |   6799MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

Map (num_proc=4):   0%|          | 0/1000 [00:00<?, ? examples/s]

===training policy===


Step,Training Loss,Validation Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                             ┃ score                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │  can't find a word to use in my life.",    │ -2.384777069091797   │
│ test a potential employee\'s ability to     │ "What is the meaning of the word           │                      │
│ solve puzzles to determine their            │ 'diamond'?", "What is the meaning of the   │                      │
│ suitability for a job. Write a Python       │ word 'diamond' and 'gold'?", "What is the  │                      │
│ script that generates a list of questions   │ meaning of the word 'gold' and             │                      │
│ that require logical reasoning to answer.   │                                            │                      │
│ Your list should include questions related  │                                            │                      │
│ to mathematical puzzles, language puzzles,  │                                            │                      │
│ logic puzzles, lateral thinking puzzles,    │                                            │                      │
│ and pattern recognition puzzles. Use the    │                                            │                      │
│ following code as a starting                │                                            │                      │
│ point:\nquestions = {\n    "Mathematical    │                                            │                      │
│ puzzles": ["If the value of x+y = 20 and    │                                            │                      │
│ x-y = 10, what is the value of x and y?",   │                                            │                      │
│ "If a pizza has a radius of 8 inches and is │                                            │                      │
│ cut into 6 equal slices, what is the area   │                                            │                      │
│ of each slice?"],\n    "Language puzzles":  │                                            │                      │
│ ["What word starts with \'e\' and ends with │                                            │                      │
│ \'e\' but only contains one letter?", "I am │                                            │                      │
│ taken from a mine, and shut up in a wooden  │                                            │                      │
│ case, from which I                          │                                            │                      │
├─────────────────────────────────────────────┼────────────────────────────────────────────┼──────────────────────┤
│ [{'content': 'Definition: In this task, you │  and environmental safety;   improve the   │ 0.3373647928237915   │
│ are given a summary for US Congressional    │ quality of life of its employees;          │                      │
│ and California state bill, your task is to  │ improve the quality of life of its         │                      │
│ generate a Title for this bill. The         │ facilities;   improve the quality of life  │                      │
│ preferred titles are under forty words and  │ of its facilities and facilities'          │                      │
│ mention the purpose of the bill.\nInput:    │                                            │                      │
│ Intelligent Technologies Initiative Act of  │                                            │                      │
│ 2015  This bill directs the Department of   │                                            │                      │
│ Transportation (DOT) to establish an        │                                            │                      │
│ Intelligent Technology Initiative to        │         

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │  can't think of any other answer. I'm sure   │ 6.525981426239014  │
│ test a potential employee\'s ability to     │ you can't think of any other answer, but I'm │                    │
│ solve puzzles to determine their            │ sure you can't think of any other way to     │                    │
│ suitability for a job. Write a Python       │ answer it. I'm sure you can't think of any   │                    │
│ script that generates a list of questions   │ other way to answer it, but I                │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["If the value of x+y = 20 and    │                                              │                    │
│ x-y = 10, what is the value of x and y?",   │                                              │                    │
│ "If a pizza has a radius of 8 inches and is │                                              │                    │
│ cut into 6 equal slices, what is the area   │                                              │                    │
│ of each slice?"],\n    "Language puzzles":  │                                              │                    │
│ ["What word starts with \'e\' and ends with │                                              │                    │
│ \'e\' but only contains one letter?", "I am │                                              │                    │
│ taken from a mine, and shut up in a wooden  │                                              │                    │
│ case, from which I                          │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │  and environmental safety.                   │ 7.23288106918335   │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ I'm sure that you're all familiar with the   │                    │
│ generate a Title for this bill. The         │ idea of a "smart city" but I'm not sure that │                    │
│ preferred titles are under forty words and  │ you're familiar with the fact that the US    │                    │
│ mention the purpose of the bill.\nInput:    │ government is not doing anything about it.   │                    │
│ Intelligent Technologies Initiative Act of  │                                              │                    │
│ 2015  This bill directs the Department of   │ I'm sure that you                            │                    │
│ Transportation (DOT) to establish an        │                                              │                    │
│ Intelligent Technology Initiative to        │         

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                             ┃ score                ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │  can't get a picture. I'm sure you can't   │ 4.414800643920898    │
│ test a potential employee\'s ability to     │ use this code, but I'm sure you can use it │                      │
│ solve puzzles to determine their            │ to get a picture. I'm sure you can't use   │                      │
│ suitability for a job. Write a Python       │ it to get a picture, but I'm sure you can  │                      │
│ script that generates a list of questions   │ use it to get a                            │                      │
│ that require logical reasoning to answer.   │                                            │                      │
│ Your list should include questions related  │                                            │                      │
│ to mathematical puzzles, language puzzles,  │                                            │                      │
│ logic puzzles, lateral thinking puzzles,    │                                            │                      │
│ and pattern recognition puzzles. Use the    │                                            │                      │
│ following code as a starting                │                                            │                      │
│ point:\nquestions = {\n    "Mathematical    │                                            │                      │
│ puzzles": ["If the value of x+y = 20 and    │                                            │                      │
│ x-y = 10, what is the value of x and y?",   │                                            │                      │
│ "If a pizza has a radius of 8 inches and is │                                            │                      │
│ cut into 6 equal slices, what is the area   │                                            │                      │
│ of each slice?"],\n    "Language puzzles":  │                                            │                      │
│ ["What word starts with \'e\' and ends with │                                            │                      │
│ \'e\' but only contains one letter?", "I am │                                            │                      │
│ taken from a mine, and shut up in a wooden  │                                            │                      │
│ case, from which I                          │                                            │                      │
├─────────────────────────────────────────────┼────────────────────────────────────────────┼──────────────────────┤
│ [{'content': 'Definition: In this task, you │  and environmental safety.                 │ 9.642169952392578    │
│ are given a summary for US Congressional    │                                            │                      │
│ and California state bill, your task is to  │ I'm sure you can't believe that I'm        │                      │
│ generate a Title for this bill. The         │ writing this, but I'm sure you can't       │                      │
│ preferred titles are under forty words and  │ believe that I'm writing this, but I'm     │                      │
│ mention the purpose of the bill.\nInput:    │ sure you can't believe that I'm writing    │                      │
│ Intelligent Technologies Initiative Act of  │ this, but I'm sure you can                 │                      │
│ 2015  This bill directs the Department of   │                                            │                      │
│ Transportation (DOT) to establish an        │                                            │                      │
│ Intelligent Technology Initiative to        │         

Step,Training Loss,Validation Loss


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │  can't answer any questions. I'm sure you    │ 7.148004531860352  │
│ test a potential employee\'s ability to     │ can't use this code, but I'm sure you can    │                    │
│ solve puzzles to determine their            │ use it to get a better answer. I'm sure you  │                    │
│ suitability for a job. Write a Python       │ can't use it to get a better answer, but I'm │                    │
│ script that generates a list of questions   │ sure you can use it to                       │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["If the value of x+y = 20 and    │                                              │                    │
│ x-y = 10, what is the value of x and y?",   │                                              │                    │
│ "If a pizza has a radius of 8 inches and is │                                              │                    │
│ cut into 6 equal slices, what is the area   │                                              │                    │
│ of each slice?"],\n    "Language puzzles":  │                                              │                    │
│ ["What word starts with \'e\' and ends with │                                              │                    │
│ \'e\' but only contains one letter?", "I am │                                              │                    │
│ taken from a mine, and shut up in a wooden  │                                              │                    │
│ case, from which I                          │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │  and environmental safety.                   │ 9.642169952392578  │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ I'm sure you can't believe that I'm writing  │                    │
│ generate a Title for this bill. The         │ this, but I'm sure you can't believe that    │                    │
│ preferred titles are under forty words and  │ I'm writing this, but I'm sure you can't     │                    │
│ mention the purpose of the bill.\nInput:    │ believe that I'm writing this, but I'm sure  │                    │
│ Intelligent Technologies Initiative Act of  │ you can                                      │                    │
│ 2015  This bill directs the Department of   │                                              │                    │
│ Transportation (DOT) to establish an        │                                              │                    │
│ Intelligent Technology Initiative to        │         

NameError: name 'config' is not defined

In [ ]:
trainer.save_model(ppo_config.output_dir)
trainer.generate_completions()

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━┓
┃ query                                       ┃ model response                               ┃ score              ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━┩
│ [{'content': 'As an HR manager, you want to │  can't answer any questions. I'm sure you    │ 7.148004531860352  │
│ test a potential employee\'s ability to     │ can't use this code, but I'm sure you can    │                    │
│ solve puzzles to determine their            │ use it to get a better answer. I'm sure you  │                    │
│ suitability for a job. Write a Python       │ can't use it to get a better answer, but I'm │                    │
│ script that generates a list of questions   │ sure you can use it to                       │                    │
│ that require logical reasoning to answer.   │                                              │                    │
│ Your list should include questions related  │                                              │                    │
│ to mathematical puzzles, language puzzles,  │                                              │                    │
│ logic puzzles, lateral thinking puzzles,    │                                              │                    │
│ and pattern recognition puzzles. Use the    │                                              │                    │
│ following code as a starting                │                                              │                    │
│ point:\nquestions = {\n    "Mathematical    │                                              │                    │
│ puzzles": ["If the value of x+y = 20 and    │                                              │                    │
│ x-y = 10, what is the value of x and y?",   │                                              │                    │
│ "If a pizza has a radius of 8 inches and is │                                              │                    │
│ cut into 6 equal slices, what is the area   │                                              │                    │
│ of each slice?"],\n    "Language puzzles":  │                                              │                    │
│ ["What word starts with \'e\' and ends with │                                              │                    │
│ \'e\' but only contains one letter?", "I am │                                              │                    │
│ taken from a mine, and shut up in a wooden  │                                              │                    │
│ case, from which I                          │                                              │                    │
├─────────────────────────────────────────────┼──────────────────────────────────────────────┼────────────────────┤
│ [{'content': 'Definition: In this task, you │  and environmental safety.                   │ 9.642169952392578  │
│ are given a summary for US Congressional    │                                              │                    │
│ and California state bill, your task is to  │ I'm sure you can't believe that I'm writing  │                    │
│ generate a Title for this bill. The         │ this, but I'm sure you can't believe that    │                    │
│ preferred titles are under forty words and  │ I'm writing this, but I'm sure you can't     │                    │
│ mention the purpose of the bill.\nInput:    │ believe that I'm writing this, but I'm sure  │                    │
│ Intelligent Technologies Initiative Act of  │ you can                                      │                    │
│ 2015  This bill directs the Department of   │                                              │                    │
│ Transportation (DOT) to establish an        │                                              │                    │
│ Intelligent Technology Initiative to        │         

In [ ]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import contextlib

# Save the optimized PPO model
policy_model.save_pretrained("./ppo_optimized_policy")
tokenizer.save_pretrained("./ppo_optimized_policy")
print("Optimized PPO model saved.")


Optimized PPO model saved.


## Testing Models on Unseen Texts

### Reward Model

In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Load the trained reward model
reward_model = AutoModelForSequenceClassification.from_pretrained("./reward_model")
tokenizer = AutoTokenizer.from_pretrained("./reward_model")

# Test inputs and responses
prompt = "What is artificial intelligence?"
responses = [
    "AI is the simulation of human intelligence in machines.",
    "AI is a field of engineering.",
]

# Tokenize and score
inputs = tokenizer([prompt] * len(responses), responses, return_tensors="pt", padding=True)
scores = reward_model(**inputs).logits.squeeze()

# Print scores
for i, response in enumerate(responses):
    print(f"Response: {response}\nScore: {scores[i].item()}\n")


OSError: Incorrect path_or_model_id: './reward_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

### PPO model

In [ ]:
from transformers import pipeline, AutoTokenizer

# Load the trained policy model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("./ppo_optimized_policy") # Load the tokenizer
generation_pipeline = pipeline("text-generation", model=policy_model, tokenizer=tokenizer)

# Example test input
test_prompts = [
    "Explain the importance of AI in healthcare.",
    "What are the ethical challenges of AI?",
]

# Generate predictions
predictions = [generation_pipeline(prompt, max_length=100, num_return_sequences=1) for prompt in test_prompts]

for i, prediction in enumerate(predictions):
    print(f"Prompt: {test_prompts[i]}\nGenerated Text: {prediction[0]['generated_text']}\n")

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Prompt: Explain the importance of AI in healthcare.
Generated Text: Explain the importance of AI in healthcare. I know people use something called "human vision" so that I can make this very important observation because I'm going to put a lot of emphasis on that because my students are doing this. They've done it for 15 years so they know it has to be the key to their survival. When something as simple as AI can actually save us, it's really inspiring to even see how smart we should be so that we care about the everyday human beings who

Prompt: What are the ethical challenges of AI?
Generated Text: What are the ethical challenges of AI?

I wouldn't advise you to use that one tool, unless you're a scientist or you're a journalist, and if you know or you don't know well, it's not that hard to decide if it's something you need to work on or if it's something you need to live with. It all depends on you. I'm not sure if someone who is familiar with AI could explain it, but I'm sure they 

In [ ]:
diverse_outputs = generation_pipeline("What is AI?", max_length=50, num_return_sequences=5)
for i, output in enumerate(diverse_outputs):
    print(f"Output {i+1}: {output['generated_text']}")


Output 1: What is AI?

I can get an answer to this, but you can't if it's too hard.

In my case, I couldn't even say that I would, because the most I would know is myself. I'm
Output 2: What is AI?

AI is the technology that determines, through careful selection they will live their lives. Humans choose their destiny in their own way.

I'll just say: there is a bit more of a deal with it, but
Output 3: What is AI?

The good news is that even if your friend thinks you're crazy, that doesn't mean you won't see something. We only have to ask ourselves, how can any other person disagree or disagree about this, and what
Output 4: What is AI?

When I first became aware of the dangers of AI, I started to believe that if AI were to develop, then we wouldn't even have civilization. After all, AI is completely artificial, but it's not a human
Output 5: What is AI?

The idea is that we are trying to move to a new medium. But what if the other country doesn't like that it's coming to America? What